<a href="https://colab.research.google.com/github/MustafaGada7/OSM2GMNS/blob/master/test_I-17/I_17_osm2gmns_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __OSM2GMNS Quickstart Notebook__

OSM2GMNS is a Python package that helps generate high-fidelity mesoscopic transportation network models from OpenStreetMap data. This enables even the smallest transportation authorities to begin analysis with a high quality network model for future analysis.


# 1. Data Collection
We rely on a few open-source projects to gather the raw data to start our analysis. Export data from your area of interest from the following sources.
 * __OpenStreetMap base map__: https://www.openstreetmap.org/export
   * __Geofabrik__: Alternative for non-rectangular Areas of Interest: http://download.geofabrik.de/
 * __Bike Transport Info__ (Optional):  https://extract.bbbike.org

For this demonstration, we'll use sample data from the ASU Transit AI Lab Github

osm2gmns also enables users to download OSM data within the region of interest using a built-in function. A region can be a state, city, or even university. On OpenStreetMap homepage, search the region name to get its unique relation id. The following example shows how to download Tempe city OSM data using function downloadOSMData.

https://www.openstreetmap.org/#map=18/33.42141/-111.93472&layers=DG

# 2. Install the osm2gmns package

In [54]:
!pip install osm2gmns==1.0.0b4

# 3. Create a network model from OpenStreetMap exports

In [55]:
!ls


I-17.osm  link.csv  node.csv  poi.csv  sample_data


In [56]:
# Import osm2gmns with the alias as "og" (easier to type)
import osm2gmns as og
input_file = 'I-17.osm'
net = og.getNetFromFile(input_file, link_types=('motorway','trunk','primary','secondary'))

#4. Modify the network model

## Example: Consolidate Complex Intersections
Intersections can have lots of nodes representing different turn lanes, crosswalks, etc.

Using og.consolidateComplexIntersections(net), we can combine nodes in intersections into a single node to make analysis faster and simpler

In [57]:
og.consolidateComplexIntersections(net, auto_identify=True)
og.generateNodeActivityInfo(net)
#og.buildMultiResolutionNets(net)
og.outputNetToCSV(net)

In [58]:
og.fillLinkAttributesWithDefaultValues(net, default_lanes=True, default_speed=True, default_capacity=True)

## Example: Points of Interest

We can connect Points of Interest (POIs) to the network model. Nodes representing the center (centroids) of POIs are generated. These nodes are connected with a link to the nearest node on the rest of the network

In [59]:
net = og.getNetFromFile('I-17.osm', link_types=('motorway','trunk','primary','secondary'), POI=True, POI_sampling_ratio=0.05)
#og.connectPOIWithNet(net)
#og.show(net)

#5. Export transportation network model

We can export our transportation network model to a CSV format for ease of use in further analysis with other systems.

og.outputNetToCSV(...) will generate two files: node.csv representing nodes, and link.csv representing links (edges) between nodes.

In [25]:
"""
Take a peek at the format of the CSVs using the Pandas library
Loading the CSV files gives us a DataFrame object which is very heavily used in
data science and analysis.

For more information on how this data is organized, check out the GMNS specification

https://github.com/zephyr-data-specs/GMNS/pull/54

"""

import pandas as pd

node_dataframe = pd.read_csv("node.csv")
node_dataframe.head()

,name,node_id,osm_node_id,ctrl_type,x_coord,y_coord,is_boundary,activity_type,poi_id,zone_id,notes
0,NaN,0,256833492,NaN,-112.112503,33.505587,NaN,NaN,NaN,NaN,NaN
1,NaN,1,41198155,NaN,-112.112158,33.507587,NaN,NaN,NaN,NaN,NaN
2,NaN,2,256833877,NaN,-112.112194,33.520326,NaN,NaN,NaN,NaN,NaN
3,NaN,3,256786661,NaN,-112.112116,33.520667,NaN,NaN,NaN,NaN,NaN
4,NaN,4,256783640,NaN,-112.112719,33.481918,NaN,NaN,NaN,NaN,NaN


In [26]:
link_dataframe = pd.read_csv("link.csv")
link_dataframe.head()

,link_id,name,osm_way_id,from_node_id,to_node_id,directed,geometry,dir_flag,length,facility_type,link_type,free_speed,free_speed_raw,lanes,capacity,allowed_uses,toll,notes
0,0,NaN,5560963,0,1,1,"LINESTRING (-112.1125032 33.5055866, -112.1124...",1,224.90,motorway,1,NaN,NaN,1.0,NaN,auto,NaN,NaN
1,1,NaN,5561615,2,3,1,"LINESTRING (-112.112194 33.5203264, -112.11211...",1,38.52,motorway,1,NaN,NaN,1.0,NaN,auto,NaN,NaN
2,2,NaN,5563898,4,5,1,"LINESTRING (-112.112719 33.481918, -112.112758...",1,100.90,motorway,1,NaN,NaN,2.0,NaN,auto,NaN,NaN
3,3,NaN,5564113,6,7,1,"LINESTRING (-112.1134364 33.4938391, -112.1133...",1,135.43,motorway,1,NaN,NaN,2.0,NaN,auto,NaN,NaN
4,4,NaN,5564691,8,9,1,"LINESTRING (-112.112487 33.5136659, -112.11254...",1,261.62,motorway,1,NaN,NaN,1.0,NaN,auto,NaN,NaN


In [ ]:
link_dataframe = pd.read_csv("poi.csv")
link_dataframe.head()

,name,poi_id,osm_way_id,osm_relation_id,building,amenity,leisure,way,geometry,centroid,area,area_ft2
0,NaN,0,129723382.0,NaN,yes,NaN,NaN,NaN,"POLYGON ((-112.1152482 33.4740804, -112.115988...",POINT (-112.11600060071525 33.47430456644522),NaN,NaN
1,NaN,1,151887275.0,NaN,yes,NaN,NaN,NaN,"POLYGON ((-112.1043231 33.5088136, -112.104353...",POINT (-112.10441453333333 33.5088005),NaN,NaN
2,NaN,2,152640513.0,NaN,yes,NaN,NaN,NaN,"POLYGON ((-112.1111441 33.4705893, -112.111411...",POINT (-112.11127500445284 33.47066053406315),NaN,NaN
3,Shamrock Foods Food Service Warehouse,3,344056968.0,NaN,retail,NaN,NaN,NaN,"POLYGON ((-112.1122096 33.4958698, -112.112079...",POINT (-112.11190905483828 33.495673188486954),NaN,NaN
4,NaN,4,370129481.0,NaN,industrial,NaN,NaN,NaN,"POLYGON ((-112.1040958 33.4672603, -112.104092...",POINT (-112.1040801996755 33.46726458802691),NaN,NaN


#6. Basic analysis with Pandas

The node and link files are in CSV format, which can be used with a wide variety of programming languages, such as R, and programs, such as Excel.

Here, we show a quick demonstration of some simple analysis using Python

In [27]:
# From the previous cells, we have pandas DataFrame objects representing the node and link data for our model
print(node_dataframe.head(), link_dataframe.head())

   name  node_id  osm_node_id ctrl_type     x_coord    y_coord  is_boundary  \
0   NaN        0    256833492       NaN -112.112503  33.505587          NaN   
1   NaN        1     41198155       NaN -112.112158  33.507587          NaN   
2   NaN        2    256833877       NaN -112.112194  33.520326          NaN   
3   NaN        3    256786661       NaN -112.112116  33.520667          NaN   
4   NaN        4    256783640       NaN -112.112719  33.481918          NaN   

   activity_type  poi_id  zone_id  notes  
0            NaN     NaN      NaN    NaN  
1            NaN     NaN      NaN    NaN  
2            NaN     NaN      NaN    NaN  
3            NaN     NaN      NaN    NaN  
4            NaN     NaN      NaN    NaN      link_id name  osm_way_id  from_node_id  to_node_id  directed  \
0        0  NaN     5560963             0           1         1   
1        1  NaN     5561615             2           3         1   
2        2  NaN     5563898             4           5         1   